In [53]:
from pathlib import Path
import pandas as pd
import sys 

# Definir la ruta base
data_path = Path('..') / 'data' / 'raw'
src_path = Path('..') / 'src' 

sys.path.append(str(src_path))
from data_cleaning_utils import *

# Leer los archivos
# Probar con diferentes separadores y encoding
salario_edad = pd.read_csv(data_path / 'salario_medio bruto_edad.csv', 
                  sep=';',  # Muchos CSVs españoles usan punto y coma
                  encoding='utf-8')

salario_comunidad = pd.read_csv(data_path / 'salario_medio_bruto_comunidad.csv',
                  sep=';',
                  encoding='utf-8')

ipv = pd.read_csv(data_path / 'ipv historico.csv',
                  sep=';',
                  encoding='utf-8')

superficie_media = pd.read_csv(data_path / 'superficie util vivienda valor medio del periodo.csv',
                  sep=';',
                  encoding='utf-8')

interes_fijo = pd.read_csv(data_path / 'tipo interes medio inicio hipotecas constituidas fijo o variable.csv',
                  sep=';',
                  encoding='utf-8')
idealista_excel = pd.read_excel(data_path / 'Copia_idealista_metros_cuadrados.xlsx')


# EXPLORACIÓN INICIAL DE DATASETS PARA ANÁLISIS HIPOTECARIO

In [3]:
# Diccionario de datasets para iterar
datasets = {
    'Salarios por Edad': salario_edad,
    'Salarios por Comunidad': salario_comunidad, 
    'IPV Histórico': ipv,
    'Excel Idealista' : idealista_excel,
    'Superficie Media': superficie_media,
    'Tipos Interés': interes_fijo
}

In [4]:
for name, df in datasets.items():
    print(f"\n📊 DATASET: {name}")
    print(f"Shape: {df.shape}")
    print(f"Columnas: {list(df.columns)}")
    print(f"Primeras 3 filas:")
    print(df.head(3))
    print(f"Tipos de datos:")
    print(df.dtypes)
    print(f"Valores nulos por columna:")
    print(df.isnull().sum())
    print("-" * 60)


📊 DATASET: Salarios por Edad
Shape: (3564, 5)
Columnas: ['Tipo de jornada', 'Grupo de edad', 'Decil', 'Periodo', 'Total']
Primeras 3 filas:
  Tipo de jornada Grupo de edad        Decil  Periodo     Total
0           Total         Total  Total decil     2023  2.273,01
1           Total         Total  Total decil     2022  2.118,79
2           Total         Total  Total decil     2021  2.076,45
Tipos de datos:
Tipo de jornada    object
Grupo de edad      object
Decil              object
Periodo             int64
Total              object
dtype: object
Valores nulos por columna:
Tipo de jornada    0
Grupo de edad      0
Decil              0
Periodo            0
Total              0
dtype: int64
------------------------------------------------------------

📊 DATASET: Salarios por Comunidad
Shape: (11880, 5)
Columnas: ['Tipo de jornada', 'Comunidades y Ciudades Autonómas', 'Decil', 'Periodo', 'Total']
Primeras 3 filas:
  Tipo de jornada Comunidades y Ciudades Autonómas        Decil  Period

# ANÁLISIS EXPLORATORIO ESPECÍFICO POR DATASET

In [4]:
# 1. SALARIOS POR EDAD - buscar grupo 25-34
print("\n🔍 SALARIOS POR EDAD - Grupos disponibles:")
if 'Grupo de edad' in salario_edad.columns:
    print(salario_edad['Grupo de edad'].unique())
elif 'grupo_edad' in salario_edad.columns:  
    print(salario_edad['grupo_edad'].unique())
else:
    print("Revisar nombre de columna de edad:", list(salario_edad.columns))


🔍 SALARIOS POR EDAD - Grupos disponibles:
['Total' 'De 16 a 24 años' 'De 25 a 34 años' 'De 35 a 44 años'
 'De 45 a 54 años' '55 y más años']


In [5]:
# 2. SALARIOS POR COMUNIDAD - CCAA disponibles
print("\n🔍 SALARIOS POR COMUNIDAD - CCAA disponibles:")
comunidad_col = None
for col in salario_comunidad.columns:
    if 'comunidad' in col.lower() or 'ccaa' in col.lower():
        comunidad_col = col
        break
        
if comunidad_col:
    print(f"Columna CCAA: {comunidad_col}")
    print(salario_comunidad[comunidad_col].unique())
else:
    print("Revisar nombres de columnas:", list(salario_comunidad.columns))



🔍 SALARIOS POR COMUNIDAD - CCAA disponibles:
Columna CCAA: Comunidades y Ciudades Autonómas
['Total Nacional' '01 Andalucía' '02 Aragón' '03 Asturias, Principado de'
 '04 Balears, Illes' '05 Canarias' '06 Cantabria' '07 Castilla y León'
 '08 Castilla - La Mancha' '09 Cataluña' '10 Comunitat Valenciana'
 '11 Extremadura' '12 Galicia' '13 Madrid, Comunidad de'
 '14 Murcia, Región de' '15 Navarra, Comunidad Foral de' '16 País Vasco'
 '17 Rioja, La' '18 Ceuta' '19 Melilla']


In [6]:
# 3. PERÍODOS TEMPORALES disponibles
print("\n📅 PERÍODOS TEMPORALES DISPONIBLES:")
for name, df in datasets.items():
    periodo_col = None
    for col in df.columns:
        if 'periodo' in col.lower() or 'año' in col.lower() or 'fecha' in col.lower():
            periodo_col = col
            break

    if periodo_col:
        años_disponibles = df[periodo_col].unique()
        años_recientes = [año for año in años_disponibles if str(año).startswith(('2022', '2023', '2024', '2025'))]
        print(f"{name}: {años_recientes if años_recientes else 'Sin datos recientes'}")
    else:
        print(f"{name}: No se encontró columna de período")



📅 PERÍODOS TEMPORALES DISPONIBLES:
Salarios por Edad: [2023, 2022]
Salarios por Comunidad: [2023, 2022]
IPV Histórico: ['2025T1', '2024T4', '2024T3', '2024T2', '2024T1', '2023T4', '2023T3', '2023T2', '2023T1', '2022T4', '2022T3', '2022T2', '2022T1']
Excel Idealista: [2025, 2024, 2023, 2022]
Superficie Media: Sin datos recientes
Tipos Interés: ['2025M03', '2025M02', '2025M01', '2024M12', '2024M11', '2024M10', '2024M09', '2024M08', '2024M07', '2024M06', '2024M05', '2024M04', '2024M03', '2024M02', '2024M01', '2023M12', '2023M11', '2023M10', '2023M09', '2023M08', '2023M07', '2023M06', '2023M05', '2023M04', '2023M03', '2023M02', '2023M01', '2022M12', '2022M11', '2022M10', '2022M09', '2022M08', '2022M07', '2022M06', '2022M05', '2022M04', '2022M03', '2022M02', '2022M01']


In [5]:
try:
    idealista_excel = pd.read_excel(data_path / 'Copia_idealista_metros_cuadrados.xlsx')
    
    print("✅ Archivo Excel cargado exitosamente")
    print(f"Shape: {idealista_excel.shape}")
    print(f"Columnas: {list(idealista_excel.columns)}")
    
    print("\nPrimeras 5 filas:")
    print(idealista_excel.head())
    
    print("\nTipos de datos:")
    print(idealista_excel.dtypes)
    
    print("\nValores nulos por columna:")
    print(idealista_excel.isnull().sum())
    
     # Verificar si hay datos de CCAA
    print("\n🗺️ VERIFICANDO CCAA DISPONIBLES:")
    # Buscar columna que contenga nombres de lugares/CCAA
    for col in idealista_excel.columns:
        if any(keyword in col.lower() for keyword in ['comunidad', 'ccaa', 'localiz', 'lugar', 'region']):
            print(f"Columna de ubicaciones: {col}")
            ubicaciones = idealista_excel[col].unique()
            print(f"Número de ubicaciones: {len(ubicaciones)}")
            print("Primeras 10 ubicaciones:")
            print(ubicaciones[:10])
            break
    
    # Verificar si hay datos de precios
    print("\n💰 VERIFICANDO PRECIOS DISPONIBLES:")
    for col in idealista_excel.columns:
        if any(keyword in col.lower() for keyword in ['precio', 'm2', 'euro', '€']):
            print(f"Columna de precios: {col}")
            # Mostrar estadísticas básicas si es numérica
            if idealista_excel[col].dtype in ['int64', 'float64']:
                print(f"Estadísticas básicas:")
                print(idealista_excel[col].describe())
            else:
                print("Primeros valores:")
                print(idealista_excel[col].head())
            break
    
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo 'Copia_idealista_metros_cuadrados.xlsx'")
    print("Verifica que el archivo esté en la carpeta 'data/raw'")
except Exception as e:
    print(f"❌ Error al cargar el archivo: {e}")


✅ Archivo Excel cargado exitosamente
Shape: (114, 8)
Columnas: ['Comunidades y Ciudades Autonómas', 'Precio€/m2', 'Variación mensual', 'Variación trimestral', 'Variación anual', 'Máximo histórico', 'Variación máximo', 'Año']

Primeras 5 filas:
  Comunidades y Ciudades Autonómas  Precio€/m2 Variación mensual  \
0                        Andalucía  2.468 €/m2             0.018   
1                           Aragón  1.469 €/m2           - 0,2 %   
2                         Asturias  1.538 €/m2             0.017   
3                         Baleares  4.905 €/m2             0.014   
4                         Canarias  3.039 €/m2             0.019   

  Variación trimestral Variación anual      Máximo histórico Variación máximo  \
0                0.053           0.143  2.468 €/m2 mayo 2025            0,0 %   
1                0.029           0.028   2.008 €/m2 mar 2007         - 26,8 %   
2                0.054           0.109   1.867 €/m2 abr 2008         - 17,6 %   
3                0.024 

Tras este análisis exploratorio nos damos cuenta que debemos realizar una limpieza de nulos, estandarización de nombres y cambio de tipos de datos. Así como realizar unos cálculos para obtener información de años necesarios. 

# LIMPIEZA DATASETS

In [6]:
# 1. LIMPIAR IDEALISTA
#limpiar precios
idealista_clean = idealista_excel.copy()


In [7]:
def limpiar_precio_m2(precio_texto):
    if pd.isna(precio_texto):
        return None
    precio_limpio = str(precio_texto).replace('€/m2', '').replace('.', '').replace(',', '.').strip()
    try:
        return int(precio_limpio)
    except:
        return None

idealista_clean['Precio m2'] = idealista_clean['Precio€/m2'].apply(limpiar_precio_m2)

print("Precios convertidos:")
print(idealista_clean[['Comunidades y Ciudades Autonómas', 'Precio€/m2', 'Precio m2']].head())


Precios convertidos:
  Comunidades y Ciudades Autonómas  Precio€/m2  Precio m2
0                        Andalucía  2.468 €/m2       2468
1                           Aragón  1.469 €/m2       1469
2                         Asturias  1.538 €/m2       1538
3                         Baleares  4.905 €/m2       4905
4                         Canarias  3.039 €/m2       3039


In [8]:
#ESTANDARIZAR NOMBRES DE CCAA
# Mapeo para unificar nombres entre datasets
mapeo_ccaa = {
    'Andalucía': '01 Andalucía',
    'Aragón': '02 Aragón', 
    'Asturias': '03 Asturias, Principado de',
    'Baleares': '04 Balears, Illes',
    'Canarias': '05 Canarias',
    'Cantabria': '06 Cantabria',
    'Castilla y León': '07 Castilla y León',
    'Castilla-La Mancha': '08 Castilla - La Mancha',
    'Cataluña': '09 Cataluña',
    'Comunidad Valenciana': '10 Comunitat Valenciana',
    'Extremadura': '11 Extremadura',
    'Galicia': '12 Galicia',
    'Madrid': '13 Madrid, Comunidad de',
    'Murcia': '14 Murcia, Región de',
    'Navarra': '15 Navarra, Comunidad Foral de',
    'País Vasco': '16 País Vasco',
    'La Rioja': '17 Rioja, La',
    'Ceuta': '18 Ceuta',
    'Melilla': '19 Melilla'
}


In [9]:
idealista_clean['CCAA_estandarizado'] = idealista_clean['Comunidades y Ciudades Autonómas'].map(mapeo_ccaa)

In [10]:
columnas_a_mantener_idealista = [
    'CCAA_estandarizado', 
    'Año', 
    'Precio m2'
]

idealista_final = idealista_clean[columnas_a_mantener_idealista].copy()


In [11]:
idealista_final = idealista_final.rename(columns={
    'CCAA_estandarizado': 'Comunidades y Ciudades Autónomas'
})
idealista_final.columns

Index(['Comunidades y Ciudades Autónomas', 'Año', 'Precio m2'], dtype='object')

EXPORT DATASET LIMPIO DE IDEALISTA
import os

Crear la carpeta data/clean si no existe
os.makedirs('data/clean', exist_ok=True)

Guardar el dataset limpio de Idealista
idealista_final.to_csv('data/clean/idealista_clean.csv', index=False)

print("Dataset de Idealista guardado exitosamente en data/clean/idealista_clean.csv")
print(f"Forma del dataset guardado: {idealista_final.shape}")

In [15]:
#IPV HISTÓRICO Y SUPERFICIE MEDIA: ELIMINAR NULOS
superficie_media=drop_nulls(superficie_media)

In [16]:
ipv=drop_nulls(ipv)

In [17]:
#LIMPIEZA DATASET IPV
# Eliminar la columna 'Total Nacional' (solo contiene 'Nacional')
ipv_clean = ipv.drop('Total Nacional', axis=1).copy()

# Renombrar columnas para mayor claridad
ipv_clean = ipv_clean.rename(columns={
    'General, vivienda nueva y de segunda mano': 'Tipo Vivienda',
    'Índices y tasas': 'Tipo Indicador'
})


In [18]:
# Limpiar y convertir Total
ipv_clean['Total'] = ipv_clean['Total'].str.replace(',', '.')
ipv_clean['Total'] = ipv_clean['Total'].astype(float)

In [19]:
#LIMPIEZA DATASET SUPERFICIE MEDIA
#ESTANDARIZAR NOMBRES DE CCAA
# Mapeo para unificar nombres entre datasets
mapeo_ccaa_2 = {
    'Andalucía': '01 Andalucía',
    'Aragón': '02 Aragón', 
    'Asturias, Principado de': '03 Asturias, Principado de',
    'Balears, Illes': '04 Balears, Illes',
    'Canarias': '05 Canarias',
    'Cantabria': '06 Cantabria',
    'Castilla y León': '07 Castilla y León',
    'Castilla - La Mancha': '08 Castilla - La Mancha',
    'Cataluña': '09 Cataluña',
    'Comunitat Valenciana': '10 Comunitat Valenciana',
    'Extremadura': '11 Extremadura',
    'Galicia': '12 Galicia',
    'Madrid, Comunidad de': '13 Madrid, Comunidad de',
    'Murcia, Región de': '14 Murcia, Región de',
    'Navarra, Comunidad Foral de': '15 Navarra, Comunidad Foral de',
    'País Vasco': '16 País Vasco',
    'Rioja, La': '17 Rioja, La',
    'Ceuta': '18 Ceuta',
    'Melilla': '19 Melilla'
}

In [20]:
superficie_media['Comunidades y Ciudades Autónomas'] = superficie_media['Comunidades y Ciudades Autónomas'].map(mapeo_ccaa_2)

/var/folders/4q/413qpbj17ks9bgdzl52cpx9m0000gn/T/ipykernel_1157/2539730385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  superficie_media['Comunidades y Ciudades Autónomas'] = superficie_media['Comunidades y Ciudades Autónomas'].map(mapeo_ccaa_2)


In [21]:
superficie_clean = superficie_media.drop('Total Nacional', axis=1).copy()

# Renombrar columnas
superficie_clean = superficie_clean.rename(columns={
    'Superficie útil de la vivienda': 'Superficie Util',
    'Tamaño del hogar': 'Tamaño Hogar',
    'periodo': 'Año'
})

In [22]:
# Limpiar y convertir Total
# Eliminar puntos de miles -  Reemplazar la coma decimal por punto
# Quitar filas donde 'Total' está vacío o solo contiene espacios
superficie_clean['Total'] = superficie_clean['Total'].str.replace('.', '', regex=False)
superficie_clean['Total'] = superficie_clean['Total'].str.replace(',', '.', regex=False)
superficie_clean['Total'] = pd.to_numeric(superficie_clean['Total'], errors='coerce')
superficie_clean = superficie_clean.dropna(subset=['Total'])

EXPORTAMOS LOS DOS CSVS LIMPIOS 
import os
os.makedirs('data/clean', exist_ok=True)
superficie_clean.to_csv('data/clean/superficie_clean.csv', index=False)
ipv_clean.to_csv('data/clean/ipv_clean.csv', index=False)

idealista_final superficie_clean y ipv_clean son los datasets limpios para trabajar en ellos 

In [37]:
overview(interes_fijo)


Shape: (1170, 4)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Tipo de interés         1170 non-null   object
 1   Naturaleza de la finca  1170 non-null   object
 2   Periodo                 1170 non-null   object
 3   Total                   1170 non-null   object
dtypes: object(4)
memory usage: 36.7+ KB
None

Tipos de datos:
Tipo de interés           object
Naturaleza de la finca    object
Periodo                   object
Total                     object
dtype: object


In [38]:
overview(salario_comunidad)


Shape: (11880, 5)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11880 entries, 0 to 11879
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Tipo de jornada                   11880 non-null  object
 1   Comunidades y Ciudades Autonómas  11880 non-null  object
 2   Decil                             11880 non-null  object
 3   Periodo                           11880 non-null  int64 
 4   Total                             11880 non-null  object
dtypes: int64(1), object(4)
memory usage: 464.2+ KB
None

Tipos de datos:
Tipo de jornada                     object
Comunidades y Ciudades Autonómas    object
Decil                               object
Periodo                              int64
Total                               object
dtype: object


In [ ]:
overview(salario_edad)


Shape: (3564, 5)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3564 entries, 0 to 3563
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Tipo de jornada  3564 non-null   object
 1   Grupo de edad    3564 non-null   object
 2   Decil            3564 non-null   object
 3   Periodo          3564 non-null   int64 
 4   Total            3564 non-null   object
dtypes: int64(1), object(4)
memory usage: 139.3+ KB
None

Tipos de datos:
Tipo de jornada    object
Grupo de edad      object
Decil              object
Periodo             int64
Total              object
dtype: object


In [57]:
salario_comunidad = limpiar_columna_total(salario_comunidad)
salario_edad = limpiar_columna_total(salario_edad)
interes_fijo = limpiar_columna_total(interes_fijo)

NameError: name 'limpiar_columna_total' is not defined

In [49]:
#LIMPIEZA SALARIO POR COMUNIDAD
# Eliminar puntos de miles y reemplazar comas decimales
salario_comunidad['Total'] = salario_comunidad['Total'].str.replace('.', '', regex=False)
salario_comunidad['Total'] = salario_comunidad['Total'].str.replace(',', '.', regex=False)

# Convertir a float y eliminar vacíos si quedan
salario_comunidad['Total'] = pd.to_numeric(salario_comunidad['Total'], errors='coerce')
salario_comunidad = salario_comunidad.dropna(subset=['Total'])

In [50]:
#LIMPIEZA SALARIO POR EDAD
salario_edad['Total'] = salario_edad['Total'].str.replace('.', '', regex=False)
salario_edad['Total'] = salario_edad['Total'].str.replace(',', '.', regex=False)

salario_edad['Total'] = pd.to_numeric(salario_edad['Total'], errors='coerce')
salario_edad = salario_edad.dropna(subset=['Total'])

In [ ]:
#LIMPIEZA INTERES FIJO 
interes_fijo['Total'] = interes_fijo['Total'].str.replace('.', '', regex=False)
interes_fijo['Total'] = interes_fijo['Total'].str.replace(',', '.', regex=False)

interes_fijo['Total'] = pd.to_numeric(interes_fijo['Total'], errors='coerce')
interes_fijo = interes_fijo.dropna(subset=['Total'])

#Elegir Naturaleza de la finca == 'Vivienda' es mejor que 'Total'
